In [1]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import pickle
from scipy.spatial import distance
from scipy import stats
import util
import time

# Data Preprocessing

In [13]:
proteomeHD_df = pd.read_csv('./data_sources/ProteomeHD_v1_1.csv')
proteomeHD_feature_names = [col for col in proteomeHD_df.columns if 'Ratio' in col]
proteomeHD_feature_matrix = proteomeHD_df[proteomeHD_feature_names].to_numpy()
# Keep only proteins quantified in at least 95 experiments 
rows_to_keep = [i for i in range(len(proteomeHD_feature_matrix)) if np.sum(~np.isnan(proteomeHD_feature_matrix[i])) >= 95]
proteomeHD_df = proteomeHD_df.iloc[rows_to_keep]
proteomeHD_gene_names = proteomeHD_df['Gene_names'].to_numpy()
proteomeHD_feature_matrix = proteomeHD_df[proteomeHD_feature_names].to_numpy()

# Pearson Correlation (Higher is better)

In [30]:
proteomeHD_pearson_corr = pd.DataFrame(proteomeHD_feature_matrix.T).corr().to_numpy()
pickle.dump(proteomeHD_pearson_corr, open("./pickle_files/proteomeHD_pearson_corr.p", "wb"))

In [33]:
proteomeHD_pearson_corr

array([[ 1.00000000e+00, -4.31410779e-01, -9.44151553e-02, ...,
         3.04476514e-01, -6.12467789e-01, -7.18215156e-04],
       [-4.31410779e-01,  1.00000000e+00, -1.59706057e-01, ...,
        -6.02307961e-01,  3.21048815e-01,  3.34175414e-01],
       [-9.44151553e-02, -1.59706057e-01,  1.00000000e+00, ...,
         4.13954433e-02, -8.89459710e-02, -2.94995178e-01],
       ...,
       [ 3.04476514e-01, -6.02307961e-01,  4.13954433e-02, ...,
         1.00000000e+00, -8.74728243e-03, -6.27747123e-02],
       [-6.12467789e-01,  3.21048815e-01, -8.89459710e-02, ...,
        -8.74728243e-03,  1.00000000e+00,  3.35174003e-01],
       [-7.18215156e-04,  3.34175414e-01, -2.94995178e-01, ...,
        -6.27747123e-02,  3.35174003e-01,  1.00000000e+00]])

# Cosine Similarity (Lower is better)

In [7]:
def calc_distance_matrix(feature_matrix,dist_func,shared_only=True):
    print(f"Calculating {dist_func.__name__}...")
    start_time = time.time()
    dist_mat = np.empty((feature_matrix.shape[0],feature_matrix.shape[0]))
    for i in range(len(feature_matrix)):
        g1 = feature_matrix[i]
        for j in range(i,len(feature_matrix)):
            g2 = feature_matrix[j]
            if shared_only:
                shared_index = np.where(~np.logical_or(np.isnan(g1),np.isnan(g2)))
                dist_mat[i,j] = dist_func(g1[shared_index],g2[shared_index])
            else:
                dist_mat[i,j] = dist_func(g1,g2)
        if i%200 == 1 and i!=1:
            util.calc_eta(start_time,i,len(feature_matrix))
    print("Done")
    return dist_mat

In [16]:
proteomeHD_cosine_sim = calc_distance_matrix(proteomeHD_feature_matrix,distance.cosine)
pickle.dump(proteomeHD_cosine_sim, open("./pickle_files/proteomeHD_cosine_sim.p", "wb"))

ETA: 1119.7055976497593 seconds
ETA: 1056.7226202719826 seconds
ETA: 1001.9879682194969 seconds
ETA: 949.5422412357974 seconds
ETA: 898.9228995691884 seconds
ETA: 849.4658327444109 seconds
ETA: 800.5243745248374 seconds
ETA: 752.0149992109463 seconds
ETA: 704.498385656019 seconds
ETA: 657.3929663490857 seconds
ETA: 610.8723600423103 seconds
ETA: 564.9084674576231 seconds
ETA: 519.5333136687229 seconds
ETA: 474.5126893068373 seconds
ETA: 429.9005548791145 seconds
ETA: 385.6745616524937 seconds
ETA: 341.89886973711083 seconds
ETA: 298.5194986645826 seconds
ETA: 255.4581479702583 seconds
ETA: 212.7289795035334 seconds
ETA: 170.24396729690636 seconds
ETA: 128.01063702979215 seconds
ETA: 86.00032440327324 seconds
ETA: 44.16654186471257 seconds
ETA: 2.495871978100718 seconds


# Euclidean Distance (Lower is better)

In [23]:
def euclidean_wrapper(g1,g2):
    if len(g1) == 0: return float('NaN')
    return np.linalg.norm(g1-g2)
proteomeHD_euclidean_dist = calc_distance_matrix(proteomeHD_feature_matrix,euclidean_wrapper)
pickle.dump(proteomeHD_euclidean_dist, open("./pickle_files/proteomeHD_euclidean_dist.p", "wb"))

Calculating euclidean_wrapper...
ETA: 255.77284706172657 seconds
ETA: 242.60632671679642 seconds
ETA: 230.76177597521942 seconds
ETA: 219.9061717022671 seconds
ETA: 209.50610541535187 seconds
ETA: 198.7533334927396 seconds
ETA: 188.11425660764382 seconds
ETA: 177.86854365049192 seconds
ETA: 167.2820610653222 seconds
ETA: 157.33678604137415 seconds
ETA: 146.5621600894157 seconds
ETA: 135.88801530896004 seconds
ETA: 125.0859380048864 seconds
ETA: 114.30197460436047 seconds
ETA: 103.63765920857038 seconds
ETA: 93.11448179725855 seconds
ETA: 83.02578673128589 seconds
ETA: 72.7065665976798 seconds
ETA: 62.42495697957846 seconds
ETA: 52.09057628128178 seconds
ETA: 41.7731706464441 seconds
ETA: 31.489648776064133 seconds
ETA: 21.2032665162728 seconds
ETA: 10.904349852094946 seconds
ETA: 0.6173137911937876 seconds
Done


# Co-observed (Higher is better)

In [34]:
def coobserved_wrapper(g1,g2):
    return len(g1)
proteomeHD_coobserved_mat = calc_distance_matrix(proteomeHD_feature_matrix,coobserved_wrapper)
pickle.dump(proteomeHD_coobserved_mat, open("./pickle_files/proteomeHD_coobserved_mat.p", "wb"))

Calculating coobserved_wrapper...
ETA: 124.99822787384488 seconds
ETA: 118.53184208192135 seconds
ETA: 112.66914652825989 seconds
ETA: 107.36052009496795 seconds
ETA: 102.28016804934262 seconds
ETA: 97.30045613043512 seconds
ETA: 92.20567569691823 seconds
ETA: 87.10891923958029 seconds
ETA: 82.04264569851772 seconds
ETA: 76.95033186808162 seconds
ETA: 71.79437222799243 seconds
ETA: 66.64800029176317 seconds
ETA: 61.68047615566056 seconds
ETA: 56.76932482666648 seconds
ETA: 51.62353338141792 seconds
ETA: 46.465574406639035 seconds
ETA: 41.26586028443684 seconds
ETA: 36.110779594097224 seconds
ETA: 30.96311382769409 seconds
ETA: 25.832864125649117 seconds
ETA: 20.706433452387362 seconds
ETA: 15.595489365435329 seconds
ETA: 10.490136948908653 seconds
ETA: 5.401252296660696 seconds
ETA: 0.3056507471012512 seconds
Done


# Spearman Correlation (Higher is better)

In [8]:
def spearman_wrapper(g1,g2):
    return stats.spearmanr(g1,g2)[0]
proteomeHD_spearman_mat = calc_distance_matrix(proteomeHD_feature_matrix,spearman_wrapper)
pickle.dump(proteomeHD_spearman_mat, open("./pickle_files/proteomeHD_spearman_corr.p", "wb"))

Calculating spearman_wrapper...
ETA: 10342.86248256911 seconds
ETA: 9693.71688305529 seconds
ETA: 9076.93428140472 seconds
ETA: 8472.717579980914 seconds
ETA: 7897.317350256097 seconds
ETA: 7338.526525958789 seconds
ETA: 6796.280493448327 seconds
ETA: 6269.063110860268 seconds
ETA: 5759.606991350089 seconds
ETA: 5268.0247539780485 seconds
ETA: 4794.031888043648 seconds
ETA: 4337.3503209471155 seconds
ETA: 3898.0682208249314 seconds
ETA: 3476.0596754659036 seconds
ETA: 3071.950366394236 seconds
ETA: 2686.4095688527614 seconds
ETA: 2319.267035350558 seconds
ETA: 1969.7468061335912 seconds
ETA: 1637.8718216865195 seconds
ETA: 1323.5027798674341 seconds
ETA: 1026.5713779644238 seconds
ETA: 747.0792282939201 seconds
ETA: 485.0082594416137 seconds
ETA: 240.35698257121314 seconds
ETA: 13.083996511892995 seconds
Done


# Write as pair list

In [19]:
def write_pair_list(names,matrix,col_names,file_path):
    assert len(names) == len(matrix)
    print("Start writing pair list")
    start_time = time.time()
    util.append_to_csv(file_path,col_names,[])
    to_write = []
    for i in range(len(names)):
        for j in range(i,len(matrix)):
            if i!=j:
                to_write.append((names[i],names[j],matrix[i,j]))
        if (i%100==1 or i==len(names)-1) and i!=1:
            util.append_to_csv(file_path,None,to_write)
            to_write.clear()
            util.calc_eta(start_time,i,len(names))
    print("Done writing pair list")

In [18]:
# Load and write proteomeHD pearson corr matrix
proteomeHD_pearson_corr = pickle.load(open("./pickle_files/proteomeHD_pearson_corr.p", "rb"))
proteomeHD_pearson_corr_dist = 1 - proteomeHD_pearson_corr
write_pair_list(proteomeHD_gene_names,proteomeHD_pearson_corr_dist,['gene_1','gene_2','pearson'],"./dist_csv/proteomeHD_pearson_corr_dist.csv")

ETA: 43.34971010566938 seconds
ETA: 41.23657262147363 seconds
ETA: 39.70954177704365 seconds
ETA: 38.3196732004979 seconds
ETA: 37.005428428421475 seconds
ETA: 35.752822239664745 seconds
ETA: 34.52129993085004 seconds
ETA: 33.34454460358352 seconds
ETA: 32.21598575012004 seconds
ETA: 31.07277051409284 seconds
ETA: 29.93349693776476 seconds
ETA: 28.814993158764487 seconds
ETA: 27.719432364602348 seconds
ETA: 26.645442948351583 seconds
ETA: 25.601035338572704 seconds
ETA: 24.57683908604295 seconds
ETA: 23.565616486564515 seconds
ETA: 22.57465677510229 seconds
ETA: 21.60456180522343 seconds
ETA: 20.64659220024921 seconds
ETA: 19.705325653642657 seconds
ETA: 18.783742529866046 seconds
ETA: 17.87961255089711 seconds
ETA: 16.99739617081991 seconds
ETA: 16.13015124608116 seconds
ETA: 15.279788271366106 seconds
ETA: 14.448652777306375 seconds
ETA: 13.634138326226111 seconds
ETA: 12.837383061185125 seconds
ETA: 12.056709149089585 seconds
ETA: 11.293105540295564 seconds
ETA: 10.546957009734678 s

In [20]:
# Load and write proteomeHD euclidean dist matrix
proteomeHD_euclidean_dist = pickle.load(open("./pickle_files/proteomeHD_euclidean_dist.p", "rb"))
write_pair_list(proteomeHD_gene_names,proteomeHD_euclidean_dist,['gene_1','gene_2','euclidean'],"./dist_csv/proteomeHD_euclidean_dist.csv")

Start writing pair list
ETA: 44.57275692779239 seconds
ETA: 41.875636371214 seconds
ETA: 40.14506993974958 seconds
ETA: 38.665231279006925 seconds
ETA: 37.32103816049542 seconds
ETA: 36.023266267062425 seconds
ETA: 35.51335524696426 seconds
ETA: 34.373063001739844 seconds
ETA: 33.226170331338935 seconds
ETA: 32.07909155892325 seconds
ETA: 30.94697980335038 seconds
ETA: 29.793883869987443 seconds
ETA: 28.649561982444393 seconds
ETA: 27.533356235675694 seconds
ETA: 26.442183745534795 seconds
ETA: 25.37955845839377 seconds
ETA: 24.33333963061136 seconds
ETA: 23.305224476358347 seconds
ETA: 22.297992582888053 seconds
ETA: 21.462885319024906 seconds
ETA: 20.48002343084743 seconds
ETA: 19.513843810650393 seconds
ETA: 18.567947507060026 seconds
ETA: 17.642390437445904 seconds
ETA: 16.734537486885127 seconds
ETA: 15.846395489253801 seconds
ETA: 14.98015268093478 seconds
ETA: 14.20727159124917 seconds
ETA: 13.36936677698019 seconds
ETA: 12.547992337032065 seconds
ETA: 11.750894071055551 seconds

In [21]:
# Load and write proteomeHD coobserved dist matrix
proteomeHD_coobserved_dist = pickle.load(open("./pickle_files/proteomeHD_coobserved_mat.p", "rb"))
write_pair_list(proteomeHD_gene_names,proteomeHD_coobserved_dist,['gene_1','gene_2','coobserved'],"./dist_csv/proteomeHD_coobserved_dist.csv")

Start writing pair list
ETA: 30.778320727962083 seconds
ETA: 28.768405359182783 seconds
ETA: 27.427693123041198 seconds
ETA: 26.335392507234417 seconds
ETA: 25.400145068140088 seconds
ETA: 24.485431342672392 seconds
ETA: 23.589054170246644 seconds
ETA: 22.73116403215387 seconds
ETA: 21.9185628478191 seconds
ETA: 21.1075585924543 seconds
ETA: 20.321148599525888 seconds
ETA: 19.560587664627214 seconds
ETA: 18.811043108178502 seconds
ETA: 18.076925532976105 seconds
ETA: 17.362082831467255 seconds
ETA: 16.67231125864366 seconds
ETA: 15.98894212107179 seconds
ETA: 15.313031786484958 seconds
ETA: 14.663403108206504 seconds
ETA: 14.00873034933339 seconds
ETA: 13.366971186148104 seconds
ETA: 12.738674393462789 seconds
ETA: 12.122481523934058 seconds
ETA: 11.626031220232333 seconds
ETA: 11.147244681839178 seconds
ETA: 10.58160038060383 seconds
ETA: 10.008072983375614 seconds
ETA: 9.44464139008854 seconds
ETA: 8.89264987444557 seconds
ETA: 8.351694310438392 seconds
ETA: 7.822535961683778 seconds

In [22]:
# Load and write proteomeHD spearman dist matrix
proteomeHD_spearman_dist = pickle.load(open("./pickle_files/proteomeHD_spearman_corr.p", "rb"))
write_pair_list(proteomeHD_gene_names,proteomeHD_spearman_dist,['gene_1','gene_2','spearman'],"./dist_csv/proteomeHD_spearman_corr_dist.csv")

Start writing pair list
ETA: 46.32666536841062 seconds
ETA: 45.30500965687766 seconds
ETA: 42.93831962762877 seconds
ETA: 41.17488799011915 seconds
ETA: 39.68723616914121 seconds
ETA: 38.20248676179451 seconds
ETA: 36.80146424134345 seconds
ETA: 35.45627310034934 seconds
ETA: 34.13894393785415 seconds
ETA: 32.86015856492293 seconds
ETA: 31.65325918899245 seconds
ETA: 30.961973122017863 seconds
ETA: 29.759771020113735 seconds
ETA: 28.632706221472272 seconds
ETA: 27.774925662707204 seconds
ETA: 26.662623732481652 seconds
ETA: 25.5442461689944 seconds
ETA: 24.58387443027782 seconds
ETA: 23.555698696780617 seconds
ETA: 22.483039261161657 seconds
ETA: 21.430892225335405 seconds
ETA: 20.40457396483432 seconds
ETA: 19.40710484530811 seconds
ETA: 18.42938711324467 seconds
ETA: 17.471196070521987 seconds
ETA: 16.539250944724955 seconds
ETA: 15.625898350613596 seconds
ETA: 14.737278159283518 seconds
ETA: 13.941306160549097 seconds
ETA: 13.08256680724701 seconds
ETA: 12.24836694544571 seconds
ETA

In [ ]:
# Create Validation file

In [12]:
proteomeHD_df.columns

Index(['Majority_protein_IDs', 'Simplified_protein_ID', 'Protein_names',
       'Gene_names', 'RatioHL_GK1_Chromatin_AL', 'RatioHL_GK1_Chromatin_CPT',
       'RatioHL_GK1_Chromatin_CR', 'RatioHL_GK1_Chromatin_HepHek',
       'RatioHL_GK1_Chromatin_hiIR', 'RatioHL_GK1_Chromatin_loIR',
       ...
       'RatioHL_PX441_E1', 'RatioHL_PX441_E2', 'RatioHL_PX441_E3',
       'RatioHL_PX441_E4', 'RatioHL_PX441_E5', 'RatioHL_PX441_F1',
       'RatioHL_PX441_F2', 'RatioHL_PX441_F3', 'RatioHL_PX441_F4',
       'RatioHL_PX441_F5'],
      dtype='object', length=298)

In [68]:
A = [1, 2, 3, 4, 5, np.NaN]
B = [2, 3, 4, 5.25, np.NaN, 100]
C = [np.NaN,3,5,np.NaN,np.NaN,np.NaN]

all_masked = ma.masked_invalid(np.array([A,B,C]))

corr_test = ma.corrcoef(all_masked)
print(corr_test)

[[1.0 8.163509630537018 1.0]
 [8.163509630537018 1.0 1.0]
 [1.0 1.0 1.0]]


In [37]:
A = [1, 2, 3, 4, 5, np.NaN]
B = [2, 3, 4, 5.25, np.NaN, 100]
C = [np.NaN,3,5,np.NaN,np.NaN,np.NaN]

nan